<a href="https://colab.research.google.com/github/Ahsrah-09/Machine-Learning/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math
from collections import Counter

from google.colab import files
files.upload()

df_buy = pd.read_csv('AllElectronics.csv')
print(df_buy)

Saving AllElectronics.csv to AllElectronics.csv
            age  income student credit_rating buys_computer
0         youth    high      no          fair            no
1         youth    high      no     excellent            no
2   middle_aged    high      no          fair           yes
3        senior  medium      no          fair           yes
4        senior     low     yes          fair           yes
5        senior     low     yes     excellent            no
6   middle_aged     low     yes     excellent           yes
7         youth  medium      no          fair            no
8         youth     low     yes          fair           yes
9        senior  medium     yes          fair           yes
10        youth  medium     yes     excellent           yes
11  middle_aged  medium      no     excellent           yes
12  middle_aged    high     yes          fair           yes
13       senior  medium      no     excellent            no


In [3]:
def entropy(probs):
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

#Function to calculate Probabilities of positive and negative examples
def entropy_of_list(a_list):
    cnt = Counter(x for x in a_list) #Count the positive and negative ex
    num_instances = len(a_list)
    #Calculate the probabilities that we required for our entropy formula
    probs = [x / num_instances for x in cnt.values()]
    #Calling entropy function for final entropy
    return entropy(probs)

In [13]:
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("Information Gain Calculation of ",split_attribute_name)
    print("target_attribute_name",target_attribute_name)
    #Grouping features of Current Attribute
    df_split = df.groupby(split_attribute_name)

    for name,group in df_split:
        print("Name: ",name)
        print("Group: ",group)

    nobs = len(df.index) * 1.0
    print("NOBS",nobs)
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    print("df_agg_ent: \n\n",df_agg_ent)
    avg_info = sum( df_agg_ent["entropy_of_list"] * df_agg_ent['<lambda_0>'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - avg_info

In [14]:
def id3(df, target_attribute_name, attribute_names, default_class=None):
    #Counting Total number of yes and no classes (Positive and negative Ex)
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])

    if len(cnt) == 1:
        return next(iter(cnt))
    # Return None for Empty Data Set
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class = max(cnt.keys())

    print("attribute_names:",attribute_names)
    gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names]

    #Separating the maximum information gain attribute after calculating the information gain
    index_of_max = gainz.index(max(gainz)) #Index of Best Attribute
    best_attr = attribute_names[index_of_max] #choosing best attribute

    #The tree is initially an empty dictionary
    tree = {best_attr:{}} # Initiate the tree with best attribute as a node
    remaining_attribute_names = [i for i in attribute_names if i != best_attr]

    for attr_val, data_subset in df.groupby(best_attr):
        subtree = id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
        tree[best_attr][attr_val] = subtree
    return tree

In [15]:
#Driver Code
#Print Attributes
attribute_names = list(df_buy.columns)
print("List of Attributes:", attribute_names)
#Remove the class attribute
attribute_names.remove('buys_computer')
print("Predictor Attributes:", attribute_names)

#Print Entropy of DataSet
total_entropy = entropy_of_list(df_buy['buys_computer'])
print("\n Total Entropy of Data Set:",total_entropy)

#Print Info gain
print('Info-gain for Age is :'+str(information_gain(df_buy, 'age', 'buys_computer')),"\n")


from pprint import pprint
tree = id3(df_buy,'buys_computer',attribute_names)

print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

List of Attributes: ['age', 'income', 'student', 'credit_rating', 'buys_computer']
Predictor Attributes: ['age', 'income', 'student', 'credit_rating']

 Total Entropy of Data Set: 0.9402859586706309
Information Gain Calculation of  age
target_attribute_name buys_computer
Name:  middle_aged
Group:              age  income student credit_rating buys_computer
2   middle_aged    high      no          fair           yes
6   middle_aged     low     yes     excellent           yes
11  middle_aged  medium      no     excellent           yes
12  middle_aged    high     yes          fair           yes
Name:  senior
Group:         age  income student credit_rating buys_computer
3   senior  medium      no          fair           yes
4   senior     low     yes          fair           yes
5   senior     low     yes     excellent            no
9   senior  medium     yes          fair           yes
13  senior  medium      no     excellent            no
Name:  youth
Group:        age  income student cr